In [1]:
import torch
import torchvision

print("torch:", torch.__version__)
print("torchvision:", torchvision.__version__)
print("CUDA available?:", torch.cuda.is_available())
print("GPU:", torch.cuda.get_device_name(0))

torch: 2.4.1+cu124
torchvision: 0.19.1+cu124
CUDA available?: True
GPU: NVIDIA A40


In [2]:
# ⚙️ 1. 패키지 설치 (필요 시)
# transformers, pillow 설치가 아직 안 되어 있으면 아래 주석을 해제하고 실행하세요.
!pip install transformers>=4.40.0 pillow


[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip


In [3]:
# 2. 라이브러리 임포트
import torch
from PIL import Image
from torchvision import transforms
from transformers import CLIPProcessor, CLIPModel

In [4]:
# 3. 디바이스 설정
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Device: {device}")

Device: cuda


In [5]:
# 6. 텍스트 전처리
# 비교할 텍스트 문장들을 리스트에 담습니다.
from transformers import CLIPTokenizerFast, CLIPImageProcessor

model_name = "openai/clip-vit-base-patch32"

# 1) 이미지 전처기: Fast 버전 사용
image_processor = CLIPImageProcessor.from_pretrained(model_name, use_fast=True)

# 2) 토크나이저: Fast 토크나이저 사용
tokenizer = CLIPTokenizerFast.from_pretrained(model_name)

# Safetensors 포맷으로 가중치를 불러오도록 설정
# torch 버전이 2.6 미만이라도 안전하게 로드됩니다.
model = CLIPModel.from_pretrained(
    model_name,
    use_safetensors=True    # ✨ 이 옵션을 추가하세요
).to(device)

# 6. 텍스트 전처리 (수정)
texts = [
    "a photo of a cat",
    "a photo of a dog",
    "a landscape with mountains"
]
# Fast 토크나이저를 이용해 토큰화 + 패딩
text_inputs = tokenizer(
    texts,
    return_tensors="pt",
    padding=True,      # 길이 맞춤
    truncation=True    # 필요 시 자르기
)
input_ids = text_inputs.input_ids.to(device)
attention_mask = text_inputs.attention_mask.to(device)

# 5. 이미지 전처리 (별도 적용)
image_path = "Golden-Retriever.jpg"
image = Image.open(image_path).convert("RGB")
image_inputs = image_processor(
    images=image,
    return_tensors="pt"
)
pixel_values = image_inputs.pixel_values.to(device)


preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/862k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/605M [00:00<?, ?B/s]

In [6]:
# 7. 임베딩 생성
with torch.no_grad():
    # 이미지 임베딩 (batch_size=1, 512차원)
    image_embeds = model.get_image_features(pixel_values=pixel_values)
    # 텍스트 임베딩 (batch_size=len(texts), 512차원)
    text_embeds = model.get_text_features(
        input_ids=input_ids,
        attention_mask=attention_mask
    )

In [7]:
# 8. 유사도 계산 (코사인 유사도)
# 먼저 임베딩을 정규화(normalize)합니다.
image_embeds = image_embeds / image_embeds.norm(p=2, dim=-1, keepdim=True)
text_embeds = text_embeds / text_embeds.norm(p=2, dim=-1, keepdim=True)

# 코사인 유사도: image_embeds @ text_embeds.T
similarity = (image_embeds @ text_embeds.T).squeeze(0)  # shape: (len(texts),)
# CPU로 옮겨서 출력
similarity = similarity.cpu().tolist()

In [8]:
# 9. 결과 출력
for text, score in zip(texts, similarity):
    print(f"‘{text}’ 유사도: {score:.4f}")

‘a photo of a cat’ 유사도: 0.1978
‘a photo of a dog’ 유사도: 0.2683
‘a landscape with mountains’ 유사도: 0.1775


In [9]:
# 10. Zero-shot 분류 예제
# CLIPModel의 로지트(로그잇) 스코어를 사용하여 가장 유사한 문장 선택
with torch.no_grad():
    outputs = model(
        pixel_values=pixel_values,
        input_ids=input_ids,
        attention_mask=attention_mask
    )
    logits_per_image = outputs.logits_per_image.squeeze(0)  # 이미지 → 텍스트 스코어
    probs = logits_per_image.softmax(dim=-1).cpu().tolist()

# 확률 기반으로 가장 높은 라벨 확인
best_idx = int(torch.argmax(logits_per_image))
print(f"\nZero-shot 분류 결과: '{texts[best_idx]}' (확률 {probs[best_idx]:.3f})")



Zero-shot 분류 결과: 'a photo of a dog' (확률 0.999)
